In [13]:
import os
import random
from PIL import Image, ImageDraw
from tqdm import tqdm  # 用于生成进度条

In [10]:
# 定义路径
small_images_dir = r"E:\ipynb\datasets\math_need_to_use"  # 小图片存放路径
class_mapping_path = r"E:\ipynb\pyhton_design\test_datasets\class.txt"  # 类别映射文件路径

# 生成类别映射
class_mapping = {class_name: idx for idx, class_name in enumerate(sorted(os.listdir(small_images_dir)))}
with open(class_mapping_path, "w") as mapping_file:
    for class_name, class_id in class_mapping.items():
        mapping_file.write(f"{class_name}\n")

# 加载小图片
small_images = []
for class_name in os.listdir(small_images_dir):
    class_path = os.path.join(small_images_dir, class_name)
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if filename.endswith(".png") or filename.endswith(".jpg"):
                with Image.open(os.path.join(class_path, filename)) as img:
                    small_images.append((img.copy(), class_mapping[class_name]))  # 保存图片和类别数字


Generating Images: 100%|██████████| 10/10 [00:00<00:00, 436.85it/s]


In [16]:
output_images_dir = r"E:\ipynb\pyhton_design\datasets\final\data_train"  # 生成的图片存放路径
output_labels_dir = r"E:\ipynb\pyhton_design\datasets\final\data_train"  # 标签文件存放路径

# 创建输出目录
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# 定义背景图片大小
background_size = (512 + random.randint(-200, 200), 512 + random.randint(-200, 200))

# 生成 10000 张图片
for img_idx in tqdm(range(60000), desc="Generating Images"):  # 使用 tqdm 添加进度条
    # 创建随机背景
    background = Image.new("RGB", background_size, (255, 255, 255))  # 白色背景
    draw = ImageDraw.Draw(background)

    annotations = []  # 用于存储标注内容

    # 随机添加小图片
    num_objects = random.randint(10, 20)  # 每张图片随机放置 5-20 个小图片
    for _ in range(num_objects):
        small_img, label_id = random.choice(small_images)  # 随机选取小图片
        scale = random.uniform(0.5, 1.5)  # 随机缩放比例
        resized_img = small_img.resize((int(45 * scale), int(45 * scale)))

        # 随机放置位置
        max_x = background_size[0] - resized_img.size[0]
        max_y = background_size[1] - resized_img.size[1]
        if max_x <= 0 or max_y <= 0:
            continue  # 如果背景太小，跳过
        x_min = random.randint(0, max_x)
        y_min = random.randint(0, max_y)
        x_max = x_min + resized_img.size[0]
        y_max = y_min + resized_img.size[1]

        # 添加小幅度的框形状变化
        delta_width = random.randint(-10, 10)  # 框宽度的随机偏差
        delta_height = random.randint(-10, 10)  # 框高度的随机偏差
        x_max += delta_width
        y_max += delta_height

        # 粘贴小图片到背景
        background.paste(resized_img, (x_min, y_min))

        # 归一化坐标
        x_center = round((x_min + x_max) / 2 / background_size[0], 6)
        y_center = round((y_min + y_max) / 2 / background_size[1], 6)
        width = round((x_max - x_min) / background_size[0], 6)
        height = round((y_max - y_min) / background_size[1], 6)

        # 添加标注（使用类别数字和归一化后的坐标）
        annotations.append(f"{label_id} {x_center} {y_center} {width} {height}")

    # 添加随机黑色噪点
    num_noise_points = random.randint(25, 125)  # 随机噪点数量
    for _ in range(num_noise_points):
        noise_x = random.randint(0, background_size[0] - 1)
        noise_y = random.randint(0, background_size[1] - 1)
        draw.point((noise_x, noise_y), fill="black")  # 在背景上绘制黑点

    # 保存图片
    image_path = os.path.join(output_images_dir, f"image_{img_idx}.jpg")
    background.save(image_path)

    # 保存标注
    label_path = os.path.join(output_labels_dir, f"image_{img_idx}.txt")
    with open(label_path, "w") as label_file:
        label_file.write("\n".join(annotations))


Generating Images: 100%|██████████| 60000/60000 [02:00<00:00, 497.48it/s]
